## Problema 1: Implementación del Método Simplex Estánda

\begin{aligned}
\text{Maximizar} \quad & Z = 3x_1 + 2x_2 + 5x_3 \\
\text{sujeto a} \quad 
& x_1 + x_2 + x_3 \le 100 \\
& 2x_1 + x_2 + x_3 \le 150 \\
& x_1 + 4x_2 + 2x_3 \le 80 \\
& x_1,\, x_2,\, x_3 \ge 0
\end{aligned}

###  Problema de forma estandar con variables de holgura:

$$ x_1 + x_2 + x_3 + s_1 = 100  $$
$$ 2x_1 + x_2 + x_3 + s_2 = 150  $$
$$ x_1 + 4x_2 + 2x_3 + s_3 = 80 $$
$$ x_1,\, x_2,\, x_3, s_1, s_2, s_3 \ge 0$$


### Implementacion del metodo simplex

In [ ]:
import numpy as np
import pandas as pd
from pyomo.environ import *

# Funcion objetivo Z = 3x1 + 2x2 + 5x3
A = np.array([3, 2, 5])   

# Coeficientes de las restricciones
B = np.array([[1, 1, 1], [2, 1, 1], [1, 4, 2]])

# Valores del lado derecho de las restricciones
C = np.array([100, 150, 80])

# Transformamos el problema inicial a forma estandar 
def transformacion_estandar(A, B, C):
    filas, columnas = B.shape
    variables_holgura = np.eye(filas)                     
    tabla_simplex = np.hstack([B, variables_holgura])    
    coeficientes = np.concatenate([A, np.zeros(filas)])  
    variables = [f"x{i+1}" for i in range(columnas)] + [f"s{i+1}" for i in range(filas)]
    return tabla_simplex, C, coeficientes, variables

# Imprimimos la tabla inicial simplex
tabla, C, coeficientes, nombres_variables = transformacion_estandar(A, B, C)
tabla_simplex = pd.DataFrame(tabla, columns = nombres_variables)
tabla_simplex["Solucion"] = C
tabla_simplex.index = [f"s{i+1}" for i in range(1, len(C)+1)]
fila_z = pd.DataFrame([np.concatenate([-coeficientes, [0]])], columns = nombres_variables + ["Solucion"], index=["Z"])
tabla_simplex = pd.concat([fila_z, tabla_simplex])
print("Tabla simplex inicial")
print(tabla_simplex.round(2))
print("")

# Variables basicas y sus valores
variables_basicas = [f"s{i+1}" for i in range(len(C))]
valores_basicos = C.copy()

# Variables no basicas y sus valores
variables_no_basicas = [f"x{i+1}" for i in range(len(A))]
valores_no_basicos = np.zeros(len(A))

# Imprimimos la solucion basica factible inicial
print("Solucion basica factible inicial")
for v, val in zip(variables_no_basicas, valores_no_basicos):
    print(f"{v} = {val:.2f}")
for v, val in zip(variables_basicas, valores_basicos):
    print(f"{v} = {val:.2f}")
print(f"Z = 0")

Tabla simplex inicial
     x1   x2   x3   s1   s2   s3  Solucion
Z  -3.0 -2.0 -5.0 -0.0 -0.0 -0.0       0.0
s2  1.0  1.0  1.0  1.0  0.0  0.0     100.0
s3  2.0  1.0  1.0  0.0  1.0  0.0     150.0
s4  1.0  4.0  2.0  0.0  0.0  1.0      80.0

Solucion basica factible inicial
x1 = 0.00
x2 = 0.00
x3 = 0.00
s1 = 100.00
s2 = 150.00
s3 = 80.00
Z = 0


In [ ]:
# Creacion del modelo 
model = ConcreteModel()

# Conjuntos
numero_variables = len(A)
numero_constantes = len(C)
model.V = RangeSet(0, numero_variables - 1)
model.C = RangeSet(0, numero_constantes - 1)

# Variable de decision
model.x = Var(model.V, domain = NonNegativeReals)

# Funcion objetivo 
model.Z = Objective(expr=sum(A[j] * model.x[j] for j in model.V), sense = maximize)

# Planteamos las restricciones
def restricciones(model, i):
    return sum(B[i][j] * model.x[j] for j in model.V) <= C[i]
model.restricciones = Constraint(model.C, rule=restricciones)

# Resolucion del modelo
solver = SolverFactory("glpk")
resultado = solver.solve(model) 

In [28]:
# Sacamos los valores optimos de las variables
valores_finales = {f"x{j+1}": value(model.x[j]) for j in model.V}
Z_opt = value(model.Z)

# Creamos una tabla de resumen
df_solucion = pd.DataFrame({"Variable": list(valores_finales.keys()),"Valor Optimo": list(valores_finales.values()),})

# Mostrar tabla
print("Solucion optima")
print(df_solucion.round(4).to_string(index=False))
print(f"\nValor optimo de la funcion objetivo: {Z_opt:.4f}")


Solucion optima
Variable  Valor Optimo
      x1       73.3333
      x2        0.0000
      x3        3.3333

Valor optimo de la funcion objetivo: 236.6667
